In [24]:
import numpy as np
import matplotlib.pyplot as plt

import gsm
import utils

def update_sail(
    sunlight_azimuth,
    sunlight_elevation,
    reflectivity,
    specularity,
    front_lambertian,
    back_lambertian,
    front_emissivity,
    back_emissivity,
    relative_billow,
    relative_tip_displacement,
):
    fig = plt.figure()
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    ax2 = fig.add_subplot(1, 2, 2)
    ax.view_init(elev=12., azim=-63)

    boom_half_length = 1
    billow = relative_billow*boom_half_length
    tip_displacement = relative_tip_displacement*boom_half_length

    vecs = gsm.get_square_sail_vectors(boom_half_length, billow, -tip_displacement, 20, True)

    utils.plot_sail(ax, vecs)

    sunlight_elevation = np.deg2rad(sunlight_elevation)
    sunlight_azimuth = np.deg2rad(sunlight_azimuth)
    sunlight_direction = spher2cart(sunlight_elevation, sunlight_azimuth)

    utils.plot_vector(ax, sunlight_direction, label="Sunlight", color=utils.YELLOW)

    J1_geo, J2_geo, J3_geo = gsm.get_geometrical_Js_from_square_sail_vectors(vecs)
    J1, J2, J3 = gsm.apply_coefficients(
        J1_geo,
        J2_geo,
        J3_geo,
        reflectivity,
        specularity,
        front_lambertian,
        back_lambertian,
        front_emissivity,
        back_emissivity,
    )
    force = -gsm.get_force(J1, J2, J3, sunlight_direction)/4
    utils.plot_vector(ax, force, label="Total Force", color=utils.BLACK)

    ax.legend(loc="upper right")

    elevation = np.linspace(0, np.pi/2, 20)
    force_mag = np.zeros_like(elevation)
    force_normal_mag = np.zeros_like(elevation)
    force_tangent_mag = np.zeros_like(elevation)
    for i, sunlight_elevation_i in enumerate(elevation):
        sunlight_direction = spher2cart(sunlight_elevation_i, 0)
        force = -gsm.get_force(J1, J2, J3, sunlight_direction)/4
        force_mag[i] = np.linalg.norm(force)
        force_normal_vec = np.dot( force, sunlight_direction ) * sunlight_direction
        force_tangent_vec = force - force_normal_vec
        force_normal_mag[i] = np.linalg.norm(force_normal_vec)
        force_tangent_mag[i] = np.linalg.norm(force_tangent_vec)

    utils.plot_force_profile(ax2, elevation, force_mag, force_normal_mag, force_tangent_mag, sunlight_elevation)

    fig.tight_layout()

def spher2cart(elevation, azimuth):
    elevation = np.pi - elevation
    x = np.sin(elevation) * np.cos(azimuth)
    y = np.sin(elevation) * np.sin(azimuth)
    z = np.cos(elevation)
    return np.array([x, y, z])

utils.setup_widgets(update_sail)




GridspecLayout(children=(Label(value='Sunlight Azimuth:', layout=Layout(grid_area='widget001')), FloatSlider(v…

Output()